<a href="https://colab.research.google.com/github/chang-ho-jeon/stock_data_anaysis/blob/main/stock_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance pandas


In [6]:
import yfinance as yf
import pandas as pd

# OHLCV 데이터를 받아오는 함수
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    # 최근 6개월 데이터를 받아옵니다. 기간은 필요에 맞게 조절 가능
    df = stock.history(period='6mo')

    # 주가 변동율 (고가 - 저가) / 종가 * 100
    df['Price_Change'] = (df['High'] - df['Low']) / df['Close'] * 100

    # Weight (거래량 / 주가변동율)
    df['Weight'] = df['Volume'] / df['Price_Change']

    # 거래량 변화 (오늘 거래량 - 전일 거래량)
    df['Volume_Change'] = df['Volume'].diff()

    # Demand 계산 (수요 및 공급 변화)
    def demand(row):
        if row['Close'] > row['Open'] and row['Volume_Change'] > 0:
            return '수요증가중'
        elif row['Close'] > row['Open'] and row['Volume_Change'] < 0:
            return '수요감소중'
        elif row['Close'] < row['Open'] and row['Volume_Change'] > 0:
            return '공급증가중'
        else:
            return '공급감소중'

    df['Demand'] = df.apply(demand, axis=1)

    # Strength 계산 (강세/약세)
    def strength(row):
        if row['Close'] > row['Open']:
            return '강세'
        else:
            return '약세'

    df['Strength'] = df.apply(strength, axis=1)

    return df[['Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change', 'Weight', 'Volume_Change', 'Demand', 'Strength']]

# 사용자로부터 티커 입력 받기
ticker = input("주식 티커를 입력하세요: ")
df = get_stock_data(ticker)
print(df)


주식 티커를 입력하세요: 005930.KS
                                   Open          High           Low  \
Date                                                                  
2024-04-23 00:00:00+09:00  76060.753642  76458.977483  75164.750000   
2024-04-24 00:00:00+09:00  77155.868818  78450.096295  76857.200938   
2024-04-25 00:00:00+09:00  76956.762799  77155.874734  75961.203125   
2024-04-26 00:00:00+09:00  77454.537443  77554.093404  76160.309954   
2024-04-29 00:00:00+09:00  77056.313600  77255.425522  75861.642071   
...                                 ...           ...           ...   
2024-10-16 00:00:00+09:00  59400.000000  60000.000000  59200.000000   
2024-10-17 00:00:00+09:00  59400.000000  60100.000000  59100.000000   
2024-10-18 00:00:00+09:00  59900.000000  60100.000000  59100.000000   
2024-10-21 00:00:00+09:00  59000.000000  59600.000000  58500.000000   
2024-10-23 00:00:00+09:00  57500.000000  60000.000000  57100.000000   

                                  Close    Volume  P

In [8]:
# 시간대 제거 (timezone-aware datetime을 timezone-unaware로 변환)
df.index = df.index.tz_localize(None)

# 엑셀로 저장하기
if not df.empty:
    file_name = f"{ticker}_stock_data.xlsx"  # 파일 이름 지정
    df.to_excel(file_name)  # 엑셀 파일로 저장
    print(f"데이터가 '{file_name}' 파일로 저장되었습니다.")
else:
    print("엑셀 파일을 저장할 데이터가 없습니다.")

데이터가 '005930.KS_stock_data.xlsx' 파일로 저장되었습니다.
